In [66]:
import pandas as pd
import tqdm
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import json
import io

SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
#json读写
def dict2json(output_dict,output_path):
    with open(output_path, "w", encoding='utf-8') as f:
      # json.dump(dict_, f) # 写为一行
      json.dump(output_dict, f, indent=2, sort_keys=True, ensure_ascii=False) # 写为多行
def json2dict(dict_path):
    with open(dict_path, encoding="utf-8") as f:
        json_file = json.load(f)
    return json_file
def mkfile(file):
    if not os.path.exists(file):
        os.makedirs(file)

In [35]:
#读入json文件
rawData = json2dict("./w2v_2.json")
#设置最大
max_word_in_sentence = 50
#预处理
text = []
vocab_dict
for key,li in rawData.items():
    if len(li)> max_word_in_sentence:
        sub_lists = [li[i:i + max_word_in_sentence] for i in range(0, len(li), max_word_in_sentence)]
        sub_lists.append(li[len(li)// max_word_in_sentence * max_word_in_sentence :])
        for sub_li in sub_lists:
            text.append(sub_li)
    else:
        text.append(li)
for li in text:
    temp = 50 - len(li)
    for i in range(temp):
        li.append(0)


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
['0896064001' '0909588006' '0901318002' '0848678001' '0611415034'
 '0824526004' '0797988002' '0903266001' '0831684001' '0884319006'
 '0762846026' '0919273004' '0919273004' '0927922002' '0927922002'
 '0786022008' '0913272003' '0889669006' '0913272003' '0889669006'
 '0871519008' '0871519008' '0871519008' '0919273004' 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [15]:
#创建词库：
temp = []
for key,li in rawData.items():
    for i in set(li):
        temp.append(i)
        
temp = set(temp)

vocab_dict = {}
flag = 0
for i in temp:
    if not i in vocab_dict.keys():
        vocab_dict[i] = flag
        flag+= 1
print(vocab_dict)


True
{0: 0, '0892703002': 1, '0759392007': 2, '0721609002': 3, '0927530004': 4, '0791120002': 5, '0868800006': 6, '0578476001': 7, '0855239002': 8, '0685816037': 9, '0918836001': 10, '0877599001': 11, '0861600001': 12, '0826508011': 13, '0908230002': 14, '0853163002': 15, '0770315026': 16, '0754238023': 17, '0876053003': 18, '0905614003': 19, '0904817001': 20, '0828854001': 21, '0907911001': 22, '0668956004': 23, '0743113001': 24, '0922625001': 25, '0757303018': 26, '0826492003': 27, '0630210008': 28, '0868751002': 29, '0869171001': 30, '0837448001': 31, '0767862005': 32, '0723084001': 33, '0863599002': 34, '0866991001': 35, '0902059002': 36, '0759423003': 37, '0903235001': 38, '0784926002': 39, '0909924006': 40, '0897940001': 41, '0551504010': 42, '0815446005': 43, '0689014002': 44, '0826701001': 45, '0759796005': 46, '0832977001': 47, '0665532002': 48, '0622026001': 49, '0892051005': 50, '0781758053': 51, '0833500002': 52, '0529942001': 53, '0780418005': 54, '0827632001': 55, '078370

In [25]:
# 创建转置词库
inverse_vocab = {li:key for key,li in vocab_dict.items()}
vocab_size = len(inverse_vocab)

In [43]:
demo = [3929, 223, 12102, 687, 8548, 11516, 14324, 13031, 3058, 13920, 10923, 2394, 2394, 13447, 13447, 7154, 10643, 7302, 10643, 7302, 15077, 15077, 15077, 2394, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
np.array(demo)

array([ 3929,   223, 12102,   687,  8548, 11516, 14324, 13031,  3058,
       13920, 10923,  2394,  2394, 13447, 13447,  7154, 10643,  7302,
       10643,  7302, 15077, 15077, 15077,  2394,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0])

In [47]:
#创建纯数字格式的词向量
sequences = []
for li in text:
    temp = []
    for word in li:
        temp.append(vocab_dict[word])
    temp = np.array(temp,dtype = object)
    sequences.append(temp)
print(type(sequences))
print(type(sequences[5]))
print(sequences[5])

<class 'list'>
<class 'numpy.ndarray'>
[13963 8660 10044 10525 10130 1069 12482 14442 12261 4784 14571 14571 1261
 1261 10781 1261 8761 9011 10635 983 983 14606 14606 11654 11654 10596 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [52]:
for seq in sequences:
  if(len(seq) != 50):
    print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

In [53]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [54]:
#需要这个
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 4832/4832 [00:47<00:00, 100.90it/s]




targets.shape: (361615,)
contexts.shape: (361615, 5)
labels.shape: (361615, 5)


In [70]:
print(targets[:5])
print(contexts[:5])
print(labels[:5])

[ 7302 15077 10643 14324  2394]
[[ 7302   387    32  2414   109]
 [15077     1    34   171    18]
 [ 7302   690  5318     5    14]
 [13031    80 13803  3125     0]
 [15077     3    13  9367     2]]
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]]


In [55]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


<BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int32, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [71]:
list(dataset.as_numpy_iterator())

[((array([13404, 10906,  4017, ..., 10504,  1102, 12583]),
   array([[ 7678,    65,     0,  4382,     4],
          [ 4810,   114,   281,  1266,  1026],
          [ 1430,     1,     9,  5801, 14983],
          ...,
          [ 6516,  1798,    31,     5, 12309],
          [ 9443,   313,     1,    99,  7960],
          [15120,   174,    55,     2,     7]], dtype=int64)),
  array([[1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         ...,
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0]], dtype=int64)),
 ((array([11739, 12956,  4105, ...,  7517, 12785,  6680]),
   array([[ 8272,   123,   416,     2,  1257],
          [12959,   330,    10,    35,     0],
          [ 5335,     0,  2355,     5,    17],
          ...,
          [ 7593,    14,    67,    43,     7],
          [ 9888,   393,     8,   155,    32],
          [  478,    12,   118,     2,  4576]], dtype=int64)),
  array([[1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0,

In [56]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int32, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [57]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [59]:
embedding_dim = 128
num_ns = 4
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [60]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [61]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
353/353 [==============================] - 9s 24ms/step - loss: 1.2028 - accuracy: 0.6648
Epoch 2/20
353/353 [==============================] - 9s 24ms/step - loss: 0.5113 - accuracy: 0.8361
Epoch 3/20
353/353 [==============================] - 9s 24ms/step - loss: 0.4170 - accuracy: 0.8452
Epoch 4/20
353/353 [==============================] - 9s 25ms/step - loss: 0.3795 - accuracy: 0.8563
Epoch 5/20
353/353 [==============================] - 9s 24ms/step - loss: 0.3428 - accuracy: 0.8716
Epoch 6/20
353/353 [==============================] - 8s 24ms/step - loss: 0.2999 - accuracy: 0.8926
Epoch 7/20
353/353 [==============================] - 9s 25ms/step - loss: 0.2521 - accuracy: 0.9176
Epoch 8/20
353/353 [==============================] - 8s 24ms/step - loss: 0.2033 - accuracy: 0.9427
Epoch 9/20
353/353 [==============================] - 8s 23ms/step - loss: 0.1579 - accuracy: 0.9643
Epoch 10/20
353/353 [==============================] - 8s 23ms/step - loss: 0.1190 - accura

In [62]:
#docs_infra: no_execute
%tensorboard --logdir logs

In [64]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = inverse_vocab

In [68]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
# out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
#   out_m.write(word + "\n")
out_v.close()
# out_m.close()

In [73]:
inverse_vocab

{0: 0,
 1: '0892703002',
 2: '0759392007',
 3: '0721609002',
 4: '0927530004',
 5: '0791120002',
 6: '0868800006',
 7: '0578476001',
 8: '0855239002',
 9: '0685816037',
 10: '0918836001',
 11: '0877599001',
 12: '0861600001',
 13: '0826508011',
 14: '0908230002',
 15: '0853163002',
 16: '0770315026',
 17: '0754238023',
 18: '0876053003',
 19: '0905614003',
 20: '0904817001',
 21: '0828854001',
 22: '0907911001',
 23: '0668956004',
 24: '0743113001',
 25: '0922625001',
 26: '0757303018',
 27: '0826492003',
 28: '0630210008',
 29: '0868751002',
 30: '0869171001',
 31: '0837448001',
 32: '0767862005',
 33: '0723084001',
 34: '0863599002',
 35: '0866991001',
 36: '0902059002',
 37: '0759423003',
 38: '0903235001',
 39: '0784926002',
 40: '0909924006',
 41: '0897940001',
 42: '0551504010',
 43: '0815446005',
 44: '0689014002',
 45: '0826701001',
 46: '0759796005',
 47: '0832977001',
 48: '0665532002',
 49: '0622026001',
 50: '0892051005',
 51: '0781758053',
 52: '0833500002',
 53: '05299420

In [75]:
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')
for index in range(len(vocab)):
    if index == 0:
        continue  # skip 0, it's padding
    word = vocab[index]
    out_m.write(word + "\n")
out_m.close()